<a href="https://colab.research.google.com/github/DDDCR/NLP/blob/NER/Resume_(CV)_Parsing_using_Spacy_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy-transformers
!pip install -U spacy

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
spacy.__version__

'3.7.2'

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

In [7]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json'))

In [8]:
len(cv_data)

200

In [9]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

2024-01-08 19:29:11.155311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 19:29:11.155361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 19:29:11.157121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 19:29:12.710169: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.

In [ ]:
cv_data[0]

In [11]:
#Preparing Training Data
def get_spacy_doc(file, data): # file is for error data
  nlp = spacy.blank('en')
  db = DocBin()

  for text,annotations in tqdm(data):
    doc = nlp.make_doc(text)
    annotations = annotations['entities']

    ents = []
    entity_indices = []

    for start, end, label in annotations:
      skip_entity = False  #check if the entities are overlap:
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue
      entity_indices = entity_indices + list(range(start, end)) #update entity_indices

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + '    ' + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [12]:
#Spliting data
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3) #0.7 as training data, 0.3 as test data

In [13]:
len(train), len(test)

(140, 60)

In [ ]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [17]:
len(db.tokens)

60

In [22]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2024-01-08 19:51:58.773649: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 19:51:58.773702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 19:51:58.774963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 19:51:59.883425: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly init

### Model Test


In [23]:
nlp = spacy.load('/content/output/model-best')

In [48]:
doc = nlp('my name is Vatteri David. I worked in Microsoft. I have 3 years of experience')
for ent in doc.ents:
  print(ent.text," --> ", ent.label_)

Vatteri David  -->  Name
Microsoft  -->  Companies worked at
3 years  -->  Years of Experience


### extract from pdf


In [38]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 17.3 MB/s eta 0:00:00


In [39]:
import sys, fitz

In [40]:
fname = '/content/CV-Parsing-using-Spacy-3/data/test/Sales-Administrator-CV-1.png'
doc = fitz.open(fname)

In [41]:
text = ""
for page in doc:
  text = text + str(page.get_text())

In [43]:
text = text.strip()

In [45]:
text = ' '.join(text.split())

In [46]:
text

'Alice Clark AI / Machine Learning Delhi, India Email me on Indeed • 20+ years of experience in data handling, design, and development • Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to data warehousing and business intelligence • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL) Willing to relocate anywhere WORK EXPERIENCE Software Engineer Microsoft – Bangalore, Karnataka January 2000 to Present 1. Microsoft Rewards Live dashboards: Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping online. Microsoft Rewards members can earn points when searching with Bing, browsing with Microsoft Edg

In [47]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text," --> ", ent.label_)

Alice Clark  -->  Name
AI / Machine Learning  -->  Designation
Delhi  -->  Location
Microsoft  -->  Companies worked at
Software Engineer  -->  Designation
Microsoft  -->  Companies worked at
Bangalore  -->  Location
Microsoft  -->  Companies worked at
Microsoft  -->  Companies worked at
Microsoft  -->  Companies worked at
Microsoft  -->  Companies worked at
Microsoft  -->  Companies worked at
Microsoft  -->  Companies worked at
Indian Institute of Technology  -->  College Name
Machine Learning, Natural Language Processing, and Big Data Handling ADDITIONAL INFORMATION Professional Skills •  -->  Skills
